In [1]:
import re
from mechanize import Browser
import pandas as pd
import numpy as np
from Bio import Entrez, SeqIO
from tqdm.auto import tqdm, trange
import os
import pickle
from urllib.error import HTTPError

In [2]:
# predict kon of DNA using http://nablab.rice.edu/nabtools/kinetics.html
# https://stackoverflow.com/questions/1555234/fill-form-values-in-a-web-page-via-a-python-script-not-testing/5685569#5685569
# https://stackoverflow.com/questions/18822865/python-mechanize-fill-textarea-and-submit-a-get-method-form
#https://stackoverflow.com/questions/36344619/getting-the-key-and-value-of-br-forms-in-mechanize
def predict_kon(seq):
    br = Browser()
    br.set_handle_robots(False)
    br.open("http://nablab.rice.edu/nabtools/kinetics.html")
    br.select_form(nr = 0)
    br['form2_seq'] = seq
    response = br.submit()
    result = response.read()
    result_lst = list(map(str, result.split()))

    value_float = float(list(filter(lambda x: 'e+0' in x, result_lst))[0][2:-1])
    
    return value_float

# get sequences from the mouse genome
def get_seq(chromGI, st, start, stop):
    # https://www.biostars.org/p/11431/   
    Entrez.email = "bei.liu@duke.com"     # Always tell NCBI who you are
    handle = Entrez.efetch(db="nucleotide", 
                           id=chromGI, 
                           rettype="fasta", 
                           strand=st, 
                           seq_start=start, 
                           seq_stop=stop)
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return record.seq 

In [5]:
# test kon prediction
# predict_kon('AATTGCCCTAAAGCC')

In [6]:
#https://www.biostars.org/p/155257/
#https://www.ncbi.nlm.nih.gov/assembly/GCF_000001635.20/
#mm19 genome according to the paper: DNA methylation on N6-adenine in mammalian embryonic stem cells
chrom_GI_lst = [372099109,
372099108,
372099107,
372099106,
372099105,
372099104,
372099103,
372099102,
372099101,
372099100,
372099099,
372099098,
372099097,
372099096,
372099095,
372099094,
372099093,
372099092,
372099091,
372099090,
372099089,
34538597]

# read m6A sequences from Wu, T.P. et al. DNA methylation on N(6)-adenine in mammalian embryonic stem cells. Nature 532, 329-33 (2016).
DNA_df = pd.read_csv('m6A_DNA_seq.csv')

In [8]:
# get DNA sequences containing m6A, 12 mer, m6A at the 6th nucleotide
seq = []
is_A = []
kon_pred_result = []

# for i in tqdm(range(len(DNA_df))):
#     loc = DNA_df.loc[i, 'Locus']
#     start = loc - 5
#     stop = loc + 6
#     st = DNA_df.loc[i, 'strand']
#     chrom_num = DNA_df.loc[i, 'Chromosome'][3:]
#     if chrom_num == 'X':
#         chromGI = 372099090
#     elif chrom_num == 'Y':
#         chromGI = 372099089
#     else:
#         chromGI = chrom_GI_lst[int(chrom_num)-1]
        
#     find_seq = str(get_seq(chromGI, st, start, stop))
        
#     seq.append(find_seq)
#     kon_pred_result.append(predict_kon(find_seq))   

In [34]:
# run kon prediction
# for i in trange(len(DNA_df)):
#     DNA_df.loc[i, 'pred_kon'] = predict_kon(DNA_df.loc[i, 'seq'])

In [66]:
# random_DNA_df = pd.read_csv('random_DNA_seq_12bp.csv')

# Duplex dG prediction

In [9]:
def get_comp(seq):
    '''
    return complementary sequence
    '''
    com = ''
    table= {'A':'T', 'C':'G', 'T':'A', 'G':'C'}
    for i in seq:
        com+=table[i]
    return com

# https://www.ebi.ac.uk/biomodels-static/tools/melting/
# The MELTING5.2.0 package is needed
# def pred_dG(seq):
#     '''
#     predict free energy (37C) from sequence
#     '''
#     os.system('/Users/Bei/Codelib/MELTING5.2.0/executable/melting -S ' + seq + ' -C '+get_comp(seq)+' -E Na=0.15 -P 0.001 -H dnadna -F 4 -nn all97 -sinMM allsanpey -ion owc2204 -O /Users/Bei/Kinetics_simulation/m6A_kinetics/Genome_m6A_kinetics_prediction/test.txt')

#     with open('test.txt') as f:
#         lines = f.readlines()
#         dH = float(lines[2].split()[2].replace(',', ''))
#         dS = float(lines[3].split()[2].replace(',', ''))
#     dG = (dH-((273.15+37)*dS))/1000
    
#     return dG

In [10]:
# run dG prediction for m6A sites
# dg_DNA_lst = []
# for i in tqdm((DNA_df['seq'].values)):
#     dg_DNA_lst.append(pred_dG(i))